In [1]:
# compile the model
!python3 setup.py build_ext --inplace

running build_ext


In [2]:
# remove console log and use file log 
import logging
logging.basicConfig(format='%(asctime)s %(message)s',level=logging.INFO)

# formatter = logging.Formatter('%(asctime)s %(message)s')

# filelogger = logging.getLogger()

# for h in filelogger.handlers:
#     filelogger.removeHandler(h)

# filelogger.setLevel(logging.INFO)
# fh = logging.FileHandler("log.log",mode='w')
# fh.setLevel(logging.DEBUG)
# fh.setFormatter(formatter)
# filelogger.addHandler(fh)

In [3]:
from n2v_r import run
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_predict,train_test_split
from sklearn.linear_model import LogisticRegression as lg
from sklearn.preprocessing import normalize,StandardScaler

/home/zhang18f/anaconda3/lib/python3.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
datapath = '/home/zhang18f/datasets/Cora/'

# import deepwalkDataIterator
# dataIterator = deepwalkDataIterator.DataIterator('%s/edges.csv' % datapath, 'cora.deepwalk')

import P2VDataIterator
dataIterator = P2VDataIterator.DataIterator('%s' % datapath)


In [5]:
import numpy as np
np.random.seed(0)

model = run(
    dataIterator,
    alg = 'n2v',
    output='n2v.emb',
    total_samples = 2e7,
    p=0.85,
    l2 = 1,
)

2019-04-16 22:17:22,527 initializing cython module
2019-04-16 22:17:22,548 cython module initialized
2019-04-16 22:17:22,550 use P2V
2019-04-16 22:17:22,551 use TF-ICF
2019-04-16 22:17:22,553 initilize the model
2019-04-16 22:17:22,554 loading data
2019-04-16 22:17:22,659 data contains 2708 papers, 1432 words
2019-04-16 22:17:22,667 pre-processing negative sampling for n2v
2019-04-16 22:17:22,670 init embeddings
2019-04-16 22:17:22,674 done
2019-04-16 22:17:22,675 building node draw table
2019-04-16 22:17:22,678 starting trainig threads with 20000000 samples
2019-04-16 22:17:25,020 progress: 5.00%, 1M samples trained, current loss 0.5056, current speed 0.43M/s, overall speed 0.43M/s, ETA: 44s
2019-04-16 22:17:26,981 progress: 10.00%, 2M samples trained, current loss 0.3515, current speed 0.51M/s, overall speed 0.46M/s, ETA: 38s
2019-04-16 22:17:28,933 progress: 15.00%, 3M samples trained, current loss 0.3280, current speed 0.51M/s, overall speed 0.48M/s, ETA: 35s
2019-04-16 22:17:30,89

In [8]:
i = open('%s/idxs.txt' % datapath)
l = open('%s/labels.txt' % datapath)
X = []
Y = []
labels = {}
for idx,label in zip(i,l):
    idx = idx.rstrip()
    X.append(model.paper(idx))
    label = label.rstrip()
    if label not in labels:
        labels[label] = len(labels)
    Y.append(labels[label])
    

X = normalize(X)
X = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X,Y,train_size=0.8,random_state=0)
Y_predict = lg(random_state=0).fit(X_train,y_train).predict(X_test)

print(classification_report(y_test,Y_predict,digits=4))

             precision    recall  f1-score   support

          0     0.6410    0.7812    0.7042        32
          1     0.7989    0.8935    0.8436       169
          2     0.8039    0.7193    0.7593        57
          3     0.9545    0.9438    0.9492        89
          4     0.7193    0.5942    0.6508        69
          5     0.8095    0.8095    0.8095        42
          6     0.8947    0.8095    0.8500        84

avg / total     0.8212    0.8192    0.8176       542

